In [14]:
from __future__ import absolute_import, division, print_function
import tensorflow as tf
import numpy as np

In [15]:
# HYPERPARAMS
num_classes = 10 # 0 to 9 digits
num_features = 784 # 28*28
learning_rate = 0.01
training_steps = 1000
batch_size = 256
display_step = 50

In [16]:
from tensorflow.keras.datasets import mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = np.array(x_train, np.float32), np.array(x_test, np.float32)

x_train, x_test = x_train.reshape([-1, num_features]), x_test.reshape([-1, num_features])
x_train, x_test = x_train / 255., x_test / 255.

In [17]:
train_data=tf.data.Dataset.from_tensor_slices((x_train,y_train))

train_data=train_data.repeat().shuffle(5000).batch(batch_size).prefetch(1)

In [19]:
W = tf.Variable(tf.ones([num_features, num_classes]), name="weight")
b = tf.Variable(tf.zeros([num_classes]), name="bias")

In [20]:
def logistic_regression(x):
    return tf.nn.softmax(tf.matmul(x, W) + b)

def cross_entropy(y_pred, y_true):
    y_true = tf.one_hot(y_true, depth=num_classes)
    y_pred = tf.clip_by_value(y_pred, 1e-9, 1.)
    return tf.reduce_mean(-tf.reduce_sum(y_true * tf.math.log(y_pred)))


In [21]:
def accuracy(y_pred, y_true):
  correct_prediction = tf.equal(tf.argmax(y_pred, 1), tf.cast(y_true, tf.int64))
  return tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

optimizer = tf.optimizers.SGD(learning_rate)

In [22]:
def run_optimization(x, y):
    with tf.GradientTape() as g:
        pred = logistic_regression(x)
        loss = cross_entropy(pred, y)
    gradients = g.gradient(loss, [W, b])
    optimizer.apply_gradients(zip(gradients, [W, b]))

In [23]:
for step, (batch_x, batch_y) in enumerate(train_data.take(training_steps), 1):
    run_optimization(batch_x, batch_y)
    if step % display_step == 0:
        pred = logistic_regression(batch_x)
        loss = cross_entropy(pred, batch_y)
        acc = accuracy(pred, batch_y)
        print("step: %i, loss: %f, accuracy: %f" % (step, loss, acc))

step: 50, loss: 718.801331, accuracy: 0.765625
step: 100, loss: 597.629883, accuracy: 0.808594
step: 150, loss: 564.334839, accuracy: 0.851562
step: 200, loss: 632.421265, accuracy: 0.832031
step: 250, loss: 609.436707, accuracy: 0.753906
step: 300, loss: 672.178833, accuracy: 0.757812
step: 350, loss: 787.262146, accuracy: 0.781250
step: 400, loss: 399.623413, accuracy: 0.882812
step: 450, loss: 576.305420, accuracy: 0.863281
step: 500, loss: 494.123566, accuracy: 0.875000
step: 550, loss: 637.472351, accuracy: 0.839844
step: 600, loss: 638.868225, accuracy: 0.726562
step: 650, loss: 474.212830, accuracy: 0.843750
step: 700, loss: 651.874878, accuracy: 0.824219
step: 750, loss: 370.935852, accuracy: 0.898438
step: 800, loss: 513.853516, accuracy: 0.875000
step: 850, loss: 605.323364, accuracy: 0.773438
step: 900, loss: 680.299866, accuracy: 0.839844
step: 950, loss: 783.452026, accuracy: 0.773438
step: 1000, loss: 559.627747, accuracy: 0.851562


In [24]:
pred = logistic_regression(x_test)
print("Test Accuracy: %f" % accuracy(pred, y_test))

Test Accuracy: 0.831600
